In [1]:
import urllib2
import re
import numpy as np
import csv
import time
print 'Module imported.'

Module imported.


In [2]:
class Links:
    def __init__(self, baseUrl):
        self.baseUrl = baseUrl
        self.volumeLinks = []
        self.pageLinks = []
        self.accessLinks = []
        self.firstLinks = []
    
    def currentPageLinks(self):
#         print 'current'
        url = self.baseUrl
        volume = Page(url)
        
        pagesOnThis = volume.getLinks()
        for item in pagesOnThis:
            self.firstLinks.append(item[0])
        for webs in self.firstLinks:
            webs = 'http://www.sciencedirect.com' + webs
            self.accessLinks.append(webs)
        return self.accessLinks
    
    def volLinks(self):
        self.currentPageLinks()
        url = self.baseUrl
        volume = Page(url)
        dozenVolumes = volume.getSections()
        for item in dozenVolumes:
            self.volumeLinks.append(item[0])
        for item in self.volumeLinks:
#             print '###Please Wait###'
            item = 'http://www.sciencedirect.com' + item
            page = Page(item)
            pages = page.getLinks()
            for links in pages:
                self.pageLinks.append(links[0])
        for webs in self.pageLinks:
            webs = 'http://www.sciencedirect.com' + webs
            self.accessLinks.append(webs)
        return self.accessLinks
#         return None
#         return self.volumeLinks
        
#     def pageLinks(self):
#         for item in self.volumeLinks:
#             item = 'http://www.sciencedirect.com/' + item
#             page = Page(item)
#             pages = page.getLinks()
#             for links in pages:
#                 self.pageLinks.append(links[0])
#             for webs in self.pageLinks:
#                 webs = 'http://www.sciencedirect.com/' + webs
# #                 print webs
                
#         return self.pageLinks
    
print 'Link class imported.'

Link class imported.


In [3]:
class Page:
    def __init__(self, baseUrl):
        self.baseUrl = baseUrl
    
    def getPage(self):
        headers = { 'User-Agent' : 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)' }  
        request = urllib2.Request(self.baseUrl, None, headers)
        try:
            response = urllib2.urlopen(request)
#             print '### Webpage get.'
            return response.read()#.decode('utf-8')
        except urllib2.URLError, e:
            if hasattr(e,"code"):
                print e.code
            if hasattr(e,"reason"):
                print e.reason
            return none
        
    def getJournal(self):
        page = self.getPage()
        pattern = re.compile('<span class="pubTitle.*?<h1>(.*?)</h1>.*?</span>',re.S)
        result = re.search(pattern,page)
        if result:
            return result.group(1).strip() #Journal name
        else:
            return None

    def getVolume(self):
        page = self.getPage()
        pattern = re.compile('<div.*?skyScraper">.*?<h2 >.*?<br>Volume (.*?), (.*?) \((.*?)\)</h2>',re.S)
        items = re.findall(pattern,page)
        if items:
            return items #[current volume #][page][year]
        else:
            return None
        
    def getLinks(self):
        page = self.getPage()
        pattern = re.compile('<li><div class="txt currentVolumes"><A HREF="(.*?)">(.*?)</A><br>(.*?)<.*?</div></li>',re.S)
        items = re.findall(pattern,page)
        if items:
            return items #[Single volume link][volume #][year]
        else:
            return None
        
    def getSections(self):
        page = self.getPage()
        pattern = re.compile('<li><A HREF="(.*?)".*?><span.*?</span><span.*?</span>(.*?)</A></li>',re.S)
        items = re.findall(pattern,page)
        if items:
            return items #[Ten volume link][year]
        else:
            return None
        
    def getDetail(self):
        page = self.getPage()
        pattern = re.compile('<li class="title.*?"><.*?">(.*?)</a></h4>.*?</li>.*?<i>(.*?)</i><.*?">(.*?)</li>',re.S)
        items = re.findall(pattern,page)
        if items:
            return items #[title][page][authors]
        else:
            return None

print 'Page class imported.'

Page class imported.


In [4]:
class Tools:
    replaceOTH = re.compile('[\xe2]|[\x80]|[\x9c]|[\xbc]|[\xe2]|[\x80]|[\x9d]|[\xa7]|[\x96]|[\xc4]|[\xb1]|[\xc4]|[\xb1]')
    replaceA = re.compile('[\xc3\xa0]|[\xc3\xa1]')
    replaceSPA = re.compile('[\xe2\x80\x93]|[\xc2\xa0]')
    replaceXML = re.compile('<.*?>')
    
    def replace(self,x):
        x = re.sub(self.replaceOTH,"",x)
        x = re.sub(self.replaceA,"a",x)
        x = re.sub(self.replaceSPA," ",x)
        x = re.sub(self.replaceXML,"",x)
        return x.strip()
    
    def issue(self,x):
        patternA = re.compile('Issue ')
        patternB = re.compile(', ')
        patternC = re.compile('Pages.*')
        x = re.sub(patternA,"",x)
        x = re.sub(patternB,"",x)
        x = re.sub(patternC,"",x)
        return x.strip()
    
    def year(self,x):
        patternA = re.compile('.* ')
        x = re.sub(patternA,"",x)
        return x.strip()

print 'Tools class imported.'

Tools class imported.


In [5]:
rank = 30
baseUrl = 'http://www.sciencedirect.com/science/journal/00472727/155'

#
totalStart = time.time()
tool = Tools()
counter = 0
journal = 0
print '===================='
print 'Initialized.'
print '====================\n'

linkStart = time.time()
# link = ['http://www.sciencedirect.com/science/journal/00221996/108','http://www.sciencedirect.com/science/journal/00221996/91']
links = Links(baseUrl)
link = links.volLinks()
linkEnd = time.time()
print '===================='
print 'Webpages:  \t', len(link)
print 'Time Taken:\t', linkEnd-linkStart, 's'
print '====================\n'

print '===================='
print 'Begin to Analyze Pages.'
print '====================\n'

for website in link:
    pageStart = time.time()
    rankArray = []
    journalArray = []
    yearArray = []
    volumeArray = []
    titleArray = []
    issueArray = []
    authorsArray = []
    authorOne = []
    authorTwo = []
    authorThree = []
    authorFour = []
    authorFive = []
    authorSix = []
    authorSeven = []
    counter = counter + 1
    year = 0
    volume = 0
    issue = 0
    #####Initialization in each cycle done#####
    page = Page(website)
    currentVolume = page.getVolume()
    for item in currentVolume:
        year = tool.year(item[2])
        volume = item[0]
        issue = tool.issue(item[1])
    
    journal = page.getJournal()
    
    detail = page.getDetail()
    for item in detail:
        titleArray.append(tool.replace(item[0]))
        authorsArray.append(tool.replace(item[2]))
        yearArray.append(year)
        issueArray.append(issue)
        volumeArray.append(volume)
        journalArray.append(journal)
        rankArray.append(rank)
    
    #####Split into different author field#####
    for author in authorsArray:
        pattern = re.compile(r', ')
        tempArray = re.split(pattern,author)
    #     print tempArray
        size = len(tempArray)
        if size == 1:
            authorOne.append(tempArray[0])
            authorTwo.append(None)
            authorThree.append(None)
            authorFour.append(None)
            authorFive.append(None)
            authorSix.append(None)
            authorSeven.append(None)
        if size == 2:
            authorOne.append(tempArray[0])
            authorTwo.append(tempArray[1])
            authorThree.append(None)
            authorFour.append(None)
            authorFive.append(None)
            authorSix.append(None)
            authorSeven.append(None)
        if size == 3:
            authorOne.append(tempArray[0])
            authorTwo.append(tempArray[1])
            authorThree.append(tempArray[2])
            authorFour.append(None)
            authorFive.append(None)
            authorSix.append(None)
            authorSeven.append(None)
        if size == 4:
            authorOne.append(tempArray[0])
            authorTwo.append(tempArray[1])
            authorThree.append(tempArray[2])
            authorFour.append(tempArray[3])
            authorFive.append(None)
            authorSix.append(None)
            authorSeven.append(None)
        if size == 5:
            authorOne.append(tempArray[0])
            authorTwo.append(tempArray[1])
            authorThree.append(tempArray[2])
            authorFour.append(tempArray[3])
            authorFive.append(tempArray[4])
            authorSix.append(None)
            authorSeven.append(None)
        if size == 6:
            authorOne.append(tempArray[0])
            authorTwo.append(tempArray[1])
            authorThree.append(tempArray[2])
            authorFour.append(tempArray[3])
            authorFive.append(tempArray[4])
            authorSix.append(tempArray[5])
            authorSeven.append(None)
        if size == 7:
            authorOne.append(tempArray[0])
            authorTwo.append(tempArray[1])
            authorThree.append(tempArray[2])
            authorFour.append(tempArray[3])
            authorFive.append(tempArray[4])
            authorSix.append(tempArray[5])
            authorSeven.append(tempArray[6])
    
    fileName = journal+'.csv'
    rows = zip(rankArray,journalArray,yearArray,volumeArray,issueArray,titleArray,authorOne,authorTwo,authorThree,authorFour,authorFive,authorSix,authorSeven)
    with open(fileName, "a") as f:
        writer = csv.writer(f)
        for row in rows:
            writer.writerow(row)
    
    pageEnd = time.time()
    print '===================='
    print 'Page:  \t', counter
    print 'Time:  \t', pageEnd-pageStart, 's'
    
print '====================\n'
totalEnd = time.time()
print '===================='
print 'Journal "', journal,'" Finished.'
print 'Time:  \t', totalEnd-totalStart, 's'
print '====================\n'

Initialized.

Webpages:  	370
Time Taken:	28.650965929 s

Begin to Analyze Pages.

Page:  	1
Time:  	3.79624700546 s
Page:  	2
Time:  	3.9823679924 s
Page:  	3
Time:  	4.00303888321 s
Page:  	4
Time:  	4.02584195137 s
Page:  	5
Time:  	3.87206697464 s
Page:  	6
Time:  	3.73612880707 s
Page:  	7
Time:  	9.3119790554 s
Page:  	8
Time:  	4.5102660656 s
Page:  	9
Time:  	4.37589287758 s
Page:  	10
Time:  	5.27394008636 s
Page:  	11
Time:  	4.26146292686 s
Page:  	12
Time:  	3.98722791672 s
Page:  	13
Time:  	4.40923714638 s
Page:  	14
Time:  	4.10040283203 s
Page:  	15
Time:  	4.19244384766 s
Page:  	16
Time:  	4.16681194305 s
Page:  	17
Time:  	4.09834909439 s
Page:  	18
Time:  	4.40956497192 s
Page:  	19
Time:  	4.27877497673 s
Page:  	20
Time:  	3.719176054 s
Page:  	21
Time:  	4.23285388947 s
Page:  	22
Time:  	4.07910609245 s
Page:  	23
Time:  	4.25964999199 s
Page:  	24
Time:  	4.14662098885 s
Page:  	25
Time:  	4.35801887512 s
Page:  	26
Time:  	3.89615893364 s
Page:  	27
Time:  	4.

Page:  	144
Time:  	4.11744117737 s
Page:  	145
Time:  	4.31825017929 s
Page:  	146
Time:  	5.60121488571 s
Page:  	147
Time:  	4.24638390541 s
Page:  	148
Time:  	4.01507687569 s
Page:  	149
Time:  	3.92726397514 s
Page:  	150
Time:  	5.66272306442 s
Page:  	151
Time:  	4.4618499279 s
Page:  	152
Time:  	4.72150802612 s
Page:  	153
Time:  	4.8889400959 s
Page:  	154
Time:  	4.41106820107 s
Page:  	155
Time:  	3.92627811432 s
Page:  	156
Time:  	3.81749200821 s
Page:  	157
Time:  	3.98543810844 s
Page:  	158
Time:  	5.00748682022 s
Page:  	159
Time:  	3.86019587517 s
Page:  	160
Time:  	4.19235086441 s
Page:  	161
Time:  	4.23704910278 s
Page:  	162
Time:  	6.4270529747 s
Page:  	163
Time:  	3.82045698166 s
Page:  	164
Time:  	4.52695393562 s
Page:  	165
Time:  	4.18095612526 s
Page:  	166
Time:  	4.35019111633 s
Page:  	167
Time:  	5.41944694519 s
Page:  	168
Time:  	4.40392112732 s
Page:  	169
Time:  	4.36009716988 s
Page:  	170
Time:  	4.34142994881 s
Page:  	171
Time:  	4.343441009

Page:  	289
Time:  	3.89139103889 s
Page:  	290
Time:  	4.11038994789 s
Page:  	291
Time:  	3.93809509277 s
Page:  	292
Time:  	4.14258313179 s
Page:  	293
Time:  	4.54530906677 s
Page:  	294
Time:  	4.2947909832 s
Page:  	295
Time:  	3.9729487896 s
Page:  	296
Time:  	4.11776208878 s
Page:  	297
Time:  	4.24170613289 s
Page:  	298
Time:  	3.96497488022 s
Page:  	299
Time:  	4.13619089127 s
Page:  	300
Time:  	4.26297903061 s
Page:  	301
Time:  	4.06680393219 s
Page:  	302
Time:  	11.2788910866 s
Page:  	303
Time:  	4.36728596687 s
Page:  	304
Time:  	5.81896400452 s
Page:  	305
Time:  	4.47111797333 s
Page:  	306
Time:  	4.83427095413 s
Page:  	307
Time:  	4.13078117371 s
Page:  	308
Time:  	4.35513782501 s
Page:  	309
Time:  	6.40678715706 s
Page:  	310
Time:  	4.37156105042 s
Page:  	311
Time:  	4.71698498726 s
Page:  	312
Time:  	14.84060812 s
Page:  	313
Time:  	18.6312329769 s
Page:  	314
Time:  	4.22566199303 s
Page:  	315
Time:  	4.21676683426 s
Page:  	316
Time:  	4.5956308841

TypeError: 'NoneType' object is not iterable

In [6]:
# #Base URL
# baseUrl = 'http://www.sciencedirect.com/science/journal/00221996/108'

# ##########
# tool = Tools()
# # yearArray = []
# # volumeArray = []
# # titleArray = []
# # authorsArray = []
# # authorOne = []
# # authorTwo = []
# # authorThree = []
# # authorFour = []
# # authorFive = []
# # authorSix = []
# # authorSeven = []
# print 'Initialized.'

# ##########
# links = Links(baseUrl)
# link = links.volLinks()


# # link = ['http://www.sciencedirect.com/science/journal/00221996/109','http://www.sciencedirect.com/science/journal/00221996/108','http://www.sciencedirect.com/science/journal/00221996/107']
# print 'Webpages:', len(link)


# counter = 0
# for website in link:
#     yearArray = []
#     volumeArray = []
#     titleArray = []
#     authorsArray = []
#     authorOne = []
#     authorTwo = []
#     authorThree = []
#     authorFour = []
#     authorFive = []
#     authorSix = []
#     authorSeven = []
#     counter = counter + 1
#     page = Page(website)
#     year = 0
#     volume = 0
#     currentVolume = page.getVolume()
#     for item in currentVolume:
#         year = item[2]
#         volume = item[0]
#     detail = page.getDetail()
#     for item in detail:
#         titleArray.append(tool.replace(item[0]))
#         authorsArray.append(tool.replace(item[2]))
#         yearArray.append(year)
#         volumeArray.append(volume)
    

#     for author in authorsArray:
#         pattern = re.compile(r', ')
#         tempArray = re.split(pattern,author)
#     #     print tempArray
#         size = len(tempArray)
#         if size == 1:
#             authorOne.append(tempArray[0])
#             authorTwo.append(None)
#             authorThree.append(None)
#             authorFour.append(None)
#             authorFive.append(None)
#             authorSix.append(None)
#             authorSeven.append(None)
#         if size == 2:
#             authorOne.append(tempArray[0])
#             authorTwo.append(tempArray[1])
#             authorThree.append(None)
#             authorFour.append(None)
#             authorFive.append(None)
#             authorSix.append(None)
#             authorSeven.append(None)
#         if size == 3:
#             authorOne.append(tempArray[0])
#             authorTwo.append(tempArray[1])
#             authorThree.append(tempArray[2])
#             authorFour.append(None)
#             authorFive.append(None)
#             authorSix.append(None)
#             authorSeven.append(None)
#         if size == 4:
#             authorOne.append(tempArray[0])
#             authorTwo.append(tempArray[1])
#             authorThree.append(tempArray[2])
#             authorFour.append(tempArray[3])
#             authorFive.append(None)
#             authorSix.append(None)
#             authorSeven.append(None)
#         if size == 5:
#             authorOne.append(tempArray[0])
#             authorTwo.append(tempArray[1])
#             authorThree.append(tempArray[2])
#             authorFour.append(tempArray[3])
#             authorFive.append(tempArray[4])
#             authorSix.append(None)
#             authorSeven.append(None)
#         if size == 6:
#             authorOne.append(tempArray[0])
#             authorTwo.append(tempArray[1])
#             authorThree.append(tempArray[2])
#             authorFour.append(tempArray[3])
#             authorFive.append(tempArray[4])
#             authorSix.append(tempArray[5])
#             authorSeven.append(None)
#         if size == 7:
#             authorOne.append(tempArray[0])
#             authorTwo.append(tempArray[1])
#             authorThree.append(tempArray[2])
#             authorFour.append(tempArray[3])
#             authorFive.append(tempArray[4])
#             authorSix.append(tempArray[5])
#             authorSeven.append(tempArray[6])
    
#     rows = zip(titleArray,yearArray,volumeArray,authorsArray,authorOne,authorTwo,authorThree)
#     with open('test3.csv', "a") as f:
#         writer = csv.writer(f)
#         for row in rows:
#             writer.writerow(row)
    
#     print counter, 'pages done.'

# # print titleArray
# # print authorsArray
# # print authorOne
# # print authorTwo
# # print authorThree
    
    
# # print 'Finished all the calculation. Start to write file.'
# # ##########Save to File##########
# # rows = zip(titleArray,yearArray,volumeArray,authorsArray,authorOne,authorTwo,authorThree)
# # with open('test3.csv', "a") as f:
# #     writer = csv.writer(f)
# #     for row in rows:
# #         writer.writerow(row)
# # print 'Done.'


Initialized.
###Please Wait###
###Please Wait###
###Please Wait###
###Please Wait###
###Please Wait###
###Please Wait###
###Please Wait###
###Please Wait###
###Please Wait###
###Please Wait###
Webpages: 222
1 pages done.
2 pages done.
3 pages done.
4 pages done.
5 pages done.
6 pages done.
7 pages done.
8 pages done.
9 pages done.
10 pages done.
11 pages done.
12 pages done.
13 pages done.
14 pages done.
15 pages done.
16 pages done.
17 pages done.
18 pages done.
19 pages done.
20 pages done.
21 pages done.
22 pages done.
23 pages done.
24 pages done.
25 pages done.
26 pages done.
27 pages done.
28 pages done.
29 pages done.
30 pages done.
31 pages done.
32 pages done.
33 pages done.
34 pages done.
35 pages done.
36 pages done.
37 pages done.
38 pages done.
39 pages done.
40 pages done.
41 pages done.
42 pages done.
43 pages done.
44 pages done.
45 pages done.
46 pages done.
47 pages done.
48 pages done.
49 pages done.
50 pages done.
51 pages done.
52 pages done.
53 pages done.
54 page

In [87]:
##########Test Case##########
# #Initialization
# test = Page(baseUrl)

# #Journal Name
# print test.getJournal()

# #Links on Current Page
# volumes = test.getLinks()
# for item in volumes:
#     print item[0],item[1],item[2]
    
# #Other Volume Links
# dozenVolumes = test.getSections()
# for item in dozenVolumes:
#     print item[0],item[1]

# #Get Current Volume
# currentVolume = test.getVolume()
# for item in currentVolume:
#     print item[0]#Volume
#     print item[1]#Issue
#     print item[2]#Year
#     print ''

# #Get Title, author, page
# detail = test.getDetail()
# for item in detail:
#     print item[0] #Title
#     print item[1] #Page
#     print item[2] #Author
#     print ''
    
# #Get # of articles
# print len(detail)

# #Links Class test
# test = Links(baseUrl)
# something = test.volLinks()
# for item in something:
#     print item  


#################


#     page = Page(item)
#     pages = page.getLinks()
#     for links in pages:
#         testpageLinks.append(links[0])
#         print links[0]


# url = 'http://www.sciencedirect.com/science/journal/00221996/109'

# headers = { 'User-Agent' : 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)' }  
# request = urllib2.Request(url, None, headers)

# try:
#     response = urllib2.urlopen(request)
# #     webpage = response.read()
#     webpage = response.read().decode('utf-8')
#     print 'Webpage get.'
# except urllib2.URLError, e:
#     if hasattr(e,"code"):
#         print e.code
#     if hasattr(e,"reason"):
#         print e.reason

# pattern = re.compile('<span class="pubTitle.*?<h1>(.*?)</h1>.*?</span>',re.S)
# result = re.search(pattern, webpage)
# print result.group(1)

In [86]:

# <span class="pubTitle" style="margin:0px;">
#     <h1>Journal of International Economics</h1>
# </span>

# <li><a href="/science/journal/aip/00221996"><span class="txtBold">Articles in Press</span></a></li>
# <li><a class = "cLink" querystr="?_zone=volIssueHeader&_origin=browse" href="/science/journal/00221996/open-access"><span class="txtBold">  Open Access articles</span></a></li>
# <li><A name="109"></A><A HREF="/science/journal/00221996/109" class="cLink volLink" queryStr="?_cHubEid=1-s2.0-S0022199617X00068" alt="Collapse Volume 109 2016" title="Collapse Volume 109 2016" aria-expanded="true" ><span class="srcPartOpened" title="Close">&nbsp;</span><span class="offscreen">Close</span>Volumes 101 - 109 (2016 - 2017)</A>
# <ol>
# <li class="selected"><div class="txt currentVolumes"><span aria-selected="true" style="color: #0156aa; font-weight: bold">Volume 109</span><span class="txtHidden"> - selected</span><br><span style="color: #749A43; font-weight: bold;">In Progress </span> (November 2017)</div><div class="accessIconBlk"><span class="dsub_openaccess_sci_dir " title="Issue contains Open Access articles" alt="Issue contains Open Access articles">&nbsp;<span class="offscreen">Issue contains Open Access articles</span></span></div></li>
# <li><div class="txt currentVolumes"><A HREF="/science/journal/00221996/108">Volume 108</A><br>pp. 1-444 (September 2017)</div><div class="accessIconBlk"><span class="dsub_openaccess_sci_dir " title="Issue contains Open Access articles" alt="Issue contains Open Access articles">&nbsp;<span class="offscreen">Issue contains Open Access articles</span></span></div></li>
# <li><div class="txt currentVolumes"><A HREF="/science/journal/00221996/108/supp/S1">Volume 108, Supplement 1</A><br>pp. S1-S106 (May 2017)<br><small>39th Annual NBER International Seminar on Macroeconomics</small></div><div class="accessIconBlk"><span class="dsprite_oaIcon" title="Open Access Issue" alt="Open Access Issue">&nbsp;<span class="offscreen">Open Access Issue</span></span></div></li>
# <li><div class="txt currentVolumes"><A HREF="/science/journal/00221996/107">Volume 107</A><br>pp. 1-184 (July 2017)</div><div class="accessIconBlk"><span class="dsub_article_sci_dir sci_dir" title="Entitled to full text" alt="Entitled to full text">&nbsp;<span class="offscreen">Entitled to full text</span></span></div></li>
# <li><div class="txt currentVolumes"><A HREF="/science/journal/00221996/106">Volume 106</A><br>pp. 1-184 (May 2017)</div><div class="accessIconBlk"><span class="dsub_article_sci_dir sci_dir" title="Entitled to full text" alt="Entitled to full text">&nbsp;<span class="offscreen">Entitled to full text</span></span></div></li>
# <li><div class="txt currentVolumes"><A HREF="/science/journal/00221996/105">Volume 105</A><br>pp. 1-230 (March 2017)</div><div class="accessIconBlk"><span class="dsub_openaccess_sci_dir " title="Issue contains Open Access articles" alt="Issue contains Open Access articles">&nbsp;<span class="offscreen">Issue contains Open Access articles</span></span></div></li>
# <li><div class="txt currentVolumes"><A HREF="/science/journal/00221996/104">Volume 104</A><br>pp. 1-182 (January 2017)</div><div class="accessIconBlk"><span class="dsub_article_sci_dir sci_dir" title="Entitled to full text" alt="Entitled to full text">&nbsp;<span class="offscreen">Entitled to full text</span></span></div></li>
# <li><div class="txt currentVolumes"><A HREF="/science/journal/00221996/103">Volume 103</A><br>pp. 1-262 (November 2016)</div><div class="accessIconBlk"><span class="dsub_article_sci_dir sci_dir" title="Entitled to full text" alt="Entitled to full text">&nbsp;<span class="offscreen">Entitled to full text</span></span></div></li>
# <li><div class="txt currentVolumes"><A HREF="/science/journal/00221996/102">Volume 102</A><br>pp. 1-326 (September 2016)</div><div class="accessIconBlk"><span class="dsub_openaccess_sci_dir " title="Issue contains Open Access articles" alt="Issue contains Open Access articles">&nbsp;<span class="offscreen">Issue contains Open Access articles</span></span></div></li>
# <li><div class="txt currentVolumes"><A HREF="/science/journal/00221996/101">Volume 101</A><br>pp. 1-180 (July 2016)</div><div class="accessIconBlk"><span class="dsub_openaccess_sci_dir " title="Issue contains Open Access articles" alt="Issue contains Open Access articles">&nbsp;<span class="offscreen">Issue contains Open Access articles</span></span></div></li>
# </ol>
# </li>
# <li><A HREF="/science/journal/00221996/100" alt="Expand Volume 100 2013" title="Expand Volume 100 2013" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 91 - 100 (2013 - 2016)</A></li>
# <li><A HREF="/science/journal/00221996/90" alt="Expand Volume 90 2010" title="Expand Volume 90 2010" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 81 - 90 (2010 - 2013)</A></li>
# <li><A HREF="/science/journal/00221996/80" alt="Expand Volume 80 2007" title="Expand Volume 80 2007" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 71 - 80 (2007 - 2010)</A></li>
# <li><A HREF="/science/journal/00221996/70" alt="Expand Volume 70 2003" title="Expand Volume 70 2003" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 61 - 70 (2003 - 2006)</A></li>
# <li><A HREF="/science/journal/00221996/60" alt="Expand Volume 60 2000" title="Expand Volume 60 2000" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 51 - 60 (2000 - 2003)</A></li>
# <li><A HREF="/science/journal/00221996/50" alt="Expand Volume 50 1996" title="Expand Volume 50 1996" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 41 - 50 (1996 - 2000)</A></li>
# <li><A HREF="/science/journal/00221996/40" alt="Expand Volume 40 1991" title="Expand Volume 40 1991" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 31 - 40 (1991 - 1996)</A></li>
# <li><A HREF="/science/journal/00221996/30" alt="Expand Volume 30 1986" title="Expand Volume 30 1986" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 21 - 30 (1986 - 1991)</A></li>
# <li><A HREF="/science/journal/00221996/20" alt="Expand Volume 20 1981" title="Expand Volume 20 1981" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 11 - 20 (1981 - 1986)</A></li>
# <li><A HREF="/science/journal/00221996/10" alt="Expand Volume 10 1971" title="Expand Volume 10 1971" aria-expanded="false" class="volLink"><span class="srcPartClosed" title="Open">&nbsp;</span><span class="offscreen">Open</span>Volumes 1 - 10 (1971 - 1980)</A></li>

